<a href="https://colab.research.google.com/github/frank-morales2020/MLxDL/blob/main/NAT_DS_TUTORIAL.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!uv pip install "nvidia-nat[all]"

In [12]:
from google.colab import userdata
api_key = userdata.get('NVIDIA_API_KEY')
import os
os.environ['NVIDIA_API_KEY'] = api_key

api_key_TAVILY_API_KEY = userdata.get('TAVILY_API_KEY')
os.environ['TAVILY_API_KEY'] = api_key_TAVILY_API_KEY

## CASE 1: Basic ReAct Agent & Local Tools

In [4]:
# ============================================================================
# DEEPSEEK-R1 MISSION DIAGNOSTICS - MANDATORY TAG HANDSHAKE
# ============================================================================
from google.colab import userdata
import os
import yaml

# 1. SETUP ENVIRONMENT
os.environ['NVIDIA_API_KEY'] = userdata.get('NVIDIA_API_KEY')

# 2. UPDATED WORKFLOW CONFIGURATION
config_dict = {
    "llms": {
        "reasoning_llm": {
            "_type": "nim",
            "model": "deepseek-ai/deepseek-r1",
            "base_url": "https://integrate.api.nvidia.com/v1",
            "api_key": os.environ['NVIDIA_API_KEY'],
            "max_tokens": 4000
        }
    },
    "functions": {
        "current_time": {
            "_type": "nat.tool/current_datetime"
        }
    },
    "workflow": {
        "_type": "react_agent",
        "llm_name": "reasoning_llm",
        "tool_names": ["current_time"],
        "max_iterations": 5,
        "verbose": True,
        "handle_parsing_errors": True,
        "parse_agent_response_max_retries": 10,
        "system_prompt": """
Analyze the mission telemetry. You have access to: {tools}
Available tool names: {tool_names}

Follow this format:
Thought: I need to use a tool? Yes
Action: [tool_name]
Action Input: {{}}
Observation: [result]
Thought: I now know the final answer.
Final Answer: [diagnosis]

TELEMETRY:
- ART-II Scrubber Power: 142W
- ART-II CO2: 5.2 ppm
""".strip()
    }
}

with open("stable_workflow.yml", "w") as f:
    yaml.dump(config_dict, f, default_flow_style=False)

# 3. EXECUTION
mission_input = "Analyze ART-II telemetry and provide a diagnosis with a timestamp."

print("🚀 STARTING MISSION - STABLE EXECUTION")
print("-" * 70)

# Direct CLI execution
!nat run --config_file stable_workflow.yml --input "{mission_input}"

🚀 STARTING MISSION - STABLE EXECUTION
----------------------------------------------------------------------
2026-01-20 10:10:00 - INFO     - nat.cli.commands.start:192 - Starting NAT from config file: 'stable_workflow.yml'

Configuration Summary:
--------------------
Workflow Type: react_agent
Number of Functions: 1
Number of Function Groups: 0
Number of LLMs: 1
Number of Embedders: 0
Number of Memory: 0
Number of Object Stores: 0
Number of Retrievers: 0
Number of TTC Strategies: 0
Number of Authentication Providers: 0

2026-01-20 10:10:23 - INFO     - nat.agent.react_agent.agent:169 - 
------------------------------
[AGENT]
Agent input: Analyze ART-II telemetry and provide a diagnosis with a timestamp.
Agent's thoughts: 


Thought: I need to use a tool? Yes  
Action: current_time  
Action Input: {'unused': 'timestamp'}  

------------------------------
2026-01-20 10:10:23 - INFO     - nat.agent.base:221 - 
------------------------------
[AGENT]
Calling tools: current_time
Tool's inpu

## CASE 2: Template Validation & Literal Escaping

In [7]:
# ============================================================================
# DEEPSEEK-R1 MISSION DIAGNOSTICS - FINAL LITERAL ESCAPING FIX
# ============================================================================
from google.colab import userdata
import os
import yaml

# 1. SETUP ENVIRONMENT
os.environ['NVIDIA_API_KEY'] = userdata.get('NVIDIA_API_KEY')
os.environ['TAVILY_API_KEY'] = userdata.get('TAVILY_API_KEY')

# 2. UPDATED WORKFLOW CONFIGURATION
# We use quadruple braces so the YAML file ends up with literal single braces.
config_dict = {
    "llms": {
        "reasoning_llm": {
            "_type": "nim",
            "model": "deepseek-ai/deepseek-r1",
            "base_url": "https://integrate.api.nvidia.com/v1",
            "api_key": os.environ['NVIDIA_API_KEY'],
            "max_tokens": 4000
        }
    },
    "functions": {
        "current_time": {
            "_type": "nat.tool/current_datetime"
        },
        "web_search": {
            "_type": "nat.plugins.langchain.tools/tavily_internet_search"
        }
    },
    "workflow": {
        "_type": "react_agent",
        "llm_name": "reasoning_llm",
        "tool_names": ["current_time", "web_search"],
        "max_iterations": 10,
        "verbose": True,
        "handle_parsing_errors": True,
        "parse_agent_response_max_retries": 10,
        "system_prompt": """
Analyze mission telemetry and cross-reference with NASA ECLSS standards.
You MUST follow this format strictly:

Thought: I need to use a tool? Yes
Action: [tool_name]
Action Input: {{"unused": "dummy"}} or {{"query": "search term"}}
Observation: [result]
...
Thought: I now know the final answer.
Final Answer: [Your diagnosis]

NASA REFERENCE DATA:
- CO2 Standard: < 3900 ppm.
- Scrubber Power Goal: <= 500 Watts.

Available Tools:
{tools}
Tool names: {tool_names}
""".strip()
    }
}

# Use default_flow_style=False to keep the YAML clean and readable for the NAT parser
with open("stable_workflow.yml", "w") as f:
    yaml.dump(config_dict, f, default_flow_style=False)

# 3. EXECUTION
mission_input = "1. Get the current timestamp. 2. Diagnose 142W / 5.2 ppm telemetry based on the provided NASA standards."

print("🚀 STARTING MISSION - TEMPLATE VALIDATION FIXED")
print("-" * 70)



!nat run --config_file stable_workflow.yml --input "{mission_input}"

🚀 STARTING MISSION - TEMPLATE VALIDATION FIXED
----------------------------------------------------------------------
2026-01-20 10:20:00 - INFO     - nat.cli.commands.start:192 - Starting NAT from config file: 'stable_workflow.yml'

Configuration Summary:
--------------------
Workflow Type: react_agent
Number of Functions: 2
Number of Function Groups: 0
Number of LLMs: 1
Number of Embedders: 0
Number of Memory: 0
Number of Object Stores: 0
Number of Retrievers: 0
Number of TTC Strategies: 0
Number of Authentication Providers: 0

2026-01-20 10:20:32 - INFO     - nat.agent.react_agent.agent:169 - 
------------------------------
[AGENT]
Agent input: 1. Get the current timestamp. 2. Diagnose 142W / 5.2 ppm telemetry based on the provided NASA standards.
Agent's thoughts: 


Thought: I need to use a tool? Yes  
Action: current_time  
Action Input: {"unused": "dummy"}  

------------------------------
2026-01-20 10:20:32 - INFO     - nat.agent.base:221 - 
------------------------------
[AGE

## CASE 3: Tavily Web Search Integration

In [8]:
# ============================================================================
# DEEPSEEK-R1 MISSION DIAGNOSTICS - TAVILY WEB SEARCH INTEGRATION
# ============================================================================
from google.colab import userdata
import os
import yaml

# 1. SETUP ENVIRONMENT
os.environ['NVIDIA_API_KEY'] = userdata.get('NVIDIA_API_KEY')
os.environ['TAVILY_API_KEY'] = userdata.get('TAVILY_API_KEY')

# 2. UPDATED WORKFLOW CONFIGURATION
config_dict = {
    "llms": {
        "reasoning_llm": {
            "_type": "nim",
            "model": "deepseek-ai/deepseek-r1",
            "base_url": "https://integrate.api.nvidia.com/v1",
            "api_key": os.environ['NVIDIA_API_KEY'],
            "max_tokens": 4000
        }
    },
    "functions": {
        "web_search": {
            "_type": "nat.plugins.langchain.tools/tavily_internet_search"
        }
    },
    "workflow": {
        "_type": "react_agent",
        "llm_name": "reasoning_llm",
        "tool_names": ["web_search"],
        "max_iterations": 8,
        "verbose": True,
        "handle_parsing_errors": True,
        "parse_agent_response_max_retries": 10,
        "system_prompt": """
Analyze mission telemetry and cross-reference with real-world NASA data.
You MUST follow this format strictly:

Thought: I need to use a tool? Yes
Action: web_search
Action Input: {{"query": "search query here"}}
Observation: [result]
...
Thought: I now know the final answer.
Final Answer: [Detailed Diagnosis]

Available Tools:
{tools}
Tool names: {tool_names}
""".strip()
    }
}

with open("tavily_workflow.yml", "w") as f:
    yaml.dump(config_dict, f, default_flow_style=False)

# 3. EXECUTION
# Directing the agent to find live mission bulletins.
mission_input = "Search for NASA Artemis II mission bulletins regarding ECLSS or scrubber status and compare with 142W / 5.2 ppm telemetry."

print("🚀 INITIATING LIVE MISSION SEARCH")
print("-" * 70)

!nat run --config_file tavily_workflow.yml --input "{mission_input}"

🚀 INITIATING LIVE MISSION SEARCH
----------------------------------------------------------------------
2026-01-20 10:23:10 - INFO     - nat.cli.commands.start:192 - Starting NAT from config file: 'tavily_workflow.yml'

Configuration Summary:
--------------------
Workflow Type: react_agent
Number of Functions: 1
Number of Function Groups: 0
Number of LLMs: 1
Number of Embedders: 0
Number of Memory: 0
Number of Object Stores: 0
Number of Retrievers: 0
Number of TTC Strategies: 0
Number of Authentication Providers: 0

2026-01-20 10:23:42 - INFO     - nat.agent.react_agent.agent:169 - 
------------------------------
[AGENT]
Agent input: Search for NASA Artemis II mission bulletins regarding ECLSS or scrubber status and compare with 142W / 5.2 ppm telemetry.
Agent's thoughts: 


**Final Answer:**  
Based on analysis of NASA Artemis II bulletins and telemetry cross-referencing:  

1. **ECLSS/Scrubber Status**: NASA’s July 2023 update confirmed Artemis II ECLSS testing, including CO2 scrubbe

## CASE 4: Stable Multi-Tool Configuration

In [14]:
# ============================================================================
# DEEPSEEK-R1 MISSION - FINAL STABLE MULTI-TOOL CONFIG
# ============================================================================
from google.colab import userdata
import os
import yaml

# 1. SETUP ENVIRONMENT
os.environ['NVIDIA_API_KEY'] = userdata.get('NVIDIA_API_KEY')
os.environ['TAVILY_API_KEY'] = userdata.get('TAVILY_API_KEY')

# 2. DEFINE THE WORKFLOW
# We use a single agent with BOTH tools. This is 100% stable in NAT 1.3.1.
config_dict = {
    "llms": {
        "reasoning_llm": {
            "_type": "nim",
            "model": "deepseek-ai/deepseek-r1",
            "base_url": "https://integrate.api.nvidia.com/v1",
            "api_key": os.environ['NVIDIA_API_KEY'],
            "max_tokens": 8000
        }
    },
    "functions": {
        "current_time": {
            "_type": "nat.tool/current_datetime"
        },
        "web_search": {
            "_type": "nat.plugins.langchain.tools/tavily_internet_search"
        }
    },
    "workflow": {
        "_type": "react_agent",
        "llm_name": "reasoning_llm",
        "tool_names": ["current_time", "web_search"],
        "max_iterations": 10,
        "verbose": True,
        "handle_parsing_errors": True,
        "parse_agent_response_max_retries": 10,
        "system_prompt": """
You are a mission diagnostic specialist and historian.
You have access to: {tools}
Available tools: {tool_names}

Analyze telemetry: 142W / 5.2 ppm.
1. Use current_time for the log timestamp.
2. Use web_search to find NASA Artemis II ECLSS benchmarks.
3. Compare telemetry to benchmarks for a final diagnosis.

Follow this format:
Thought: I need to [reason]
Action: [tool_name]
Action Input: {{"unused": "dummy"}} OR {{"query": "..."}}
Observation: [result]
...
Final Answer: [Detailed Report]
""".strip()
    }
}

with open("stable_workflow.yml", "w") as f:
    yaml.dump(config_dict, f, default_flow_style=False)

# 3. EXECUTION
mission_input = "Execute the collaborative diagnostic of Mission ART-II."

print("🚀 STARTING FINAL STABLE MISSION")
print("-" * 70)



!nat run --config_file stable_workflow.yml --input "{mission_input}"

🚀 STARTING FINAL STABLE MISSION
----------------------------------------------------------------------
2026-01-20 10:34:28 - INFO     - nat.cli.commands.start:192 - Starting NAT from config file: 'stable_workflow.yml'

Configuration Summary:
--------------------
Workflow Type: react_agent
Number of Functions: 2
Number of Function Groups: 0
Number of LLMs: 1
Number of Embedders: 0
Number of Memory: 0
Number of Object Stores: 0
Number of Retrievers: 0
Number of TTC Strategies: 0
Number of Authentication Providers: 0

2026-01-20 10:35:05 - INFO     - nat.agent.react_agent.agent:169 - 
------------------------------
[AGENT]
Agent input: Execute the collaborative diagnostic of Mission ART-II.
Agent's thoughts: 


Thought: I need to log the current timestamp for the diagnostic report.
Action: current_time
Action Input: {"unused": "dummy"}

------------------------------
2026-01-20 10:35:05 - INFO     - nat.agent.base:221 - 
------------------------------
[AGENT]
Calling tools: current_time
T

## CASE 5: One-Shot Finalization & Code Execution

In [21]:
# ============================================================================
# DEEPSEEK-R1 MISSION - ONE-SHOT FINALIZATION
# ============================================================================
from google.colab import userdata
import os
import yaml

# 1. SETUP ENVIRONMENT
os.environ['NVIDIA_API_KEY'] = userdata.get('NVIDIA_API_KEY')
os.environ['TAVILY_API_KEY'] = userdata.get('TAVILY_API_KEY')

# 2. DEFINE THE WORKFLOW
config_dict = {
    "llms": {
        "reasoning_llm": {
            "_type": "nim",
            "model": "deepseek-ai/deepseek-r1",
            "base_url": "https://integrate.api.nvidia.com/v1",
            "api_key": os.environ['NVIDIA_API_KEY'],
            "max_tokens": 4000
        }
    },
    "functions": {
        "current_time": { "_type": "nat.tool/current_datetime" },
        "web_search": { "_type": "nat.plugins.langchain.tools/tavily_internet_search" },
        "python_engine": {
            "_type": "nat.plugins.langchain.tools/code_generation",
            "llm_name": "reasoning_llm"
        }
    },
    "workflow": {
        "_type": "react_agent",
        "llm_name": "reasoning_llm",
        "tool_names": ["current_time", "web_search", "python_engine"],
        "max_iterations": 5, # REDUCED: Force termination before loop
        "verbose": True,
        "handle_parsing_errors": True,
        "system_prompt": """
You are a Lead Aerospace Engineer AI.
CRITICAL: After using python_engine once, you MUST stop and provide the Final Answer immediately.

Follow this plain text format:
Thought: [reasoning]
Action: [tool_name]
Action Input: {{"query": "..."}} or {{"unused": "dummy"}}
Observation: [result]
...
Final Answer: [Consolidated Report]

MISSION:
1. Log current timestamp.
2. Search NASA Artemis II updates for cabin CO2 benchmarks.
3. Use python_engine to calculate fuel: (0.000001 * 50 * 86400 * 26000) / 3000

You have access to: {tools}
Available tools: {tool_names}
""".strip()
    }
}

with open("final_mission.yml", "w") as f:
    yaml.dump(config_dict, f, default_flow_style=False)

# 3. EXECUTION
print("🚀 STARTING ONE-SHOT MISSION")
print("-" * 70)

!nat run --config_file final_mission.yml --input "Conduct the ART-II diagnostic and calculate station-keeping for 50m."

🚀 STARTING ONE-SHOT MISSION
----------------------------------------------------------------------
2026-01-20 11:00:11 - INFO     - nat.cli.commands.start:192 - Starting NAT from config file: 'final_mission.yml'
2026-01-20 11:00:11 - WARNING  - nat.profiler.utils:137 - Discovered frameworks: {<LLMFrameworkEnum.LANGCHAIN: 'langchain'>} in function code_generation_tool by inspecting source. It is recommended and more reliable to instead add the used LLMFrameworkEnum types in the framework_wrappers argument when calling @register_function.
2026-01-20 11:00:14 - INFO     - nat.plugins.langchain.tools.code_generation_tool:43 - Initializing code generation tool
Getting tool LLM from config
2026-01-20 11:00:14 - INFO     - nat.plugins.langchain.tools.code_generation_tool:54 - Filling tool's prompt variable from config
2026-01-20 11:00:14 - INFO     - nat.plugins.langchain.tools.code_generation_tool:57 - Initialized code generation tool

Configuration Summary:
--------------------
Workflow Typ